In [8]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score, 
                             mean_absolute_percentage_error)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import os
import subprocess
import sys
from datetime import datetime

print("="*70)
print("PHASE 2: SUPERVISED MACHINE LEARNING (REGRESSION)")
print("="*70)
print("\nAll required libraries loaded successfully!")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

PHASE 2: SUPERVISED MACHINE LEARNING (REGRESSION)

All required libraries loaded successfully!
Timestamp: 2025-12-04 00:30:12


# Phase 2: Supervised Machine Learning - Complete Regression Analysis

**Team Member:** Omar Khaled  
**Objective:** Build and evaluate multiple regression models for price prediction with comprehensive analysis

## Overview
This notebook implements:
- 6 regression models (XGBoost, Random Forest, Gradient Boosting, Linear Regression, Ridge, Lasso)
- Advanced metrics (RMSE, MAE, R², MAPE, Quantile Loss)
- Spatial-Temporal cross-validation
- SHAP-based interpretability analysis
- Performance analysis by temporal and spatial characteristics
- Comprehensive model comparison and deployment recommendations

In [9]:
# ==================== SECTION 1: DATA LOADING ====================
print("\n" + "="*70)
print("SECTION 1: DATA LOADING")
print("="*70)

# Load the separate train and test datasets
train_data_path = './data/processed_train_phase1.csv'
test_data_path = './data/processed_test_phase1.csv'

try:
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    print(f"\n✓ Train data loaded from: {train_data_path}")
    print(f"  Shape: {train_df.shape} (Samples: {train_df.shape[0]}, Features: {train_df.shape[1]})")
    print(f"\n✓ Test data loaded from: {test_data_path}")
    print(f"  Shape: {test_df.shape} (Samples: {test_df.shape[0]}, Features: {test_df.shape[1]})")
except FileNotFoundError as e:
    print(f"Error: {e}")
    raise

# Display dataset info
print("\n--- Train Dataset Head ---")
print(train_df.head())

print("\n--- Train Dataset Info ---")
print(train_df.info())

print("\n--- Train Dataset Statistics ---")
print(train_df.describe())


SECTION 1: DATA LOADING

✓ Train data loaded from: ./data/processed_train_phase1.csv
  Shape: (77013, 15) (Samples: 77013, Features: 15)

✓ Test data loaded from: ./data/processed_test_phase1.csv
  Shape: (19254, 15) (Samples: 19254, Features: 15)

--- Train Dataset Head ---
   passenger_count  trip_distance  pickup_longitude  pickup_latitude  \
0              5.0       1.270718         -0.981244        -0.899854   
1              1.0      -0.016575          0.432948         0.323212   
2              0.0      -0.585635         -0.015630         0.819417   
3              0.0      -0.381215          0.190997        -0.002930   
4              5.0       2.988950         -0.856830        -0.506725   

   dropoff_longitude  dropoff_latitude  fare_amount  tip_amount      hour  \
0          -0.642984         -2.451618     0.785714   -0.547945 -1.444444   
1          -0.369988          0.045149    -0.071429   -0.091324 -1.444444   
2           0.111439          1.006468    -0.785714   -0.09

In [10]:
# ==================== SECTION 2: DATA PREPARATION ====================
print("\n" + "="*70)
print("SECTION 2: FEATURE & TARGET SETUP")
print("="*70)

# Define target variable
target_col = 'total_amount'
print(f"\n✓ Target variable: {target_col}")

# Define features - exclude target and its components
exclude_cols = [target_col, 'fare_amount', 'tip_amount']
feature_cols = [col for col in train_df.columns if col not in exclude_cols]

print(f"✓ Excluded columns: {exclude_cols}")
print(f"✓ Number of features: {len(feature_cols)}")
print(f"\nFeature columns:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

# Create train/test splits
X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test = test_df[feature_cols]
y_test = test_df[target_col]

print("\n--- Data Shapes ---")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Check for missing values
print("\n--- Missing Values ---")
missing_train = X_train.isnull().sum().sum()
missing_test = X_test.isnull().sum().sum()
print(f"Missing values in X_train: {missing_train}")
print(f"Missing values in X_test: {missing_test}")

print("\n✓ Data preparation complete!")


SECTION 2: FEATURE & TARGET SETUP

✓ Target variable: total_amount
✓ Excluded columns: ['total_amount', 'fare_amount', 'tip_amount']
✓ Number of features: 12

Feature columns:
  1. passenger_count
  2. trip_distance
  3. pickup_longitude
  4. pickup_latitude
  5. dropoff_longitude
  6. dropoff_latitude
  7. hour
  8. day
  9. month
  10. weekday
  11. is_weekend
  12. is_rush_hour

--- Data Shapes ---
X_train shape: (77013, 12)
y_train shape: (77013,)
X_test shape: (19254, 12)
y_test shape: (19254,)

--- Missing Values ---
Missing values in X_train: 0
Missing values in X_test: 0

✓ Data preparation complete!


In [11]:
# ==================== SECTION 3: MODEL TRAINING ====================
print("\n" + "="*70)
print("SECTION 3: TRAINING 6 REGRESSION MODELS")
print("="*70)

# ==================== MODEL 1: XGBoost Regressor ====================
print("\n--- Model 1: XGBoost Regressor ---")
xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

print("Training XGBoost...")
xgb_reg.fit(X_train, y_train)
y_pred_xgb = xgb_reg.predict(X_test)

rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"✓ XGBoost Training Complete")
print(f"  RMSE: {rmse_xgb:.4f}")
print(f"  MAE:  {mae_xgb:.4f}")
print(f"  R²:   {r2_xgb:.4f}")


SECTION 3: TRAINING 6 REGRESSION MODELS

--- Model 1: XGBoost Regressor ---
Training XGBoost...
✓ XGBoost Training Complete
  RMSE: 2.7744
  MAE:  1.6614
  R²:   0.9326
✓ XGBoost Training Complete
  RMSE: 2.7744
  MAE:  1.6614
  R²:   0.9326


In [12]:
# ==================== MODEL 2: Random Forest Regressor ====================
print("\n--- Model 2: Random Forest Regressor ---")
rf_reg = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

print("Training Random Forest...")
rf_reg.fit(X_train, y_train)
y_pred_rf = rf_reg.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"✓ Random Forest Training Complete")
print(f"  RMSE: {rmse_rf:.4f}")
print(f"  MAE:  {mae_rf:.4f}")
print(f"  R²:   {r2_rf:.4f}")

# Extract feature importance for RF
feature_importance_rf = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_reg.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features (Random Forest):")
print(feature_importance_rf.head(10))


--- Model 2: Random Forest Regressor ---
Training Random Forest...
✓ Random Forest Training Complete
  RMSE: 2.8280
  MAE:  1.7217
  R²:   0.9300

Top 10 Most Important Features (Random Forest):
              feature  importance
1       trip_distance    0.939264
6                hour    0.013637
5    dropoff_latitude    0.013274
4   dropoff_longitude    0.012480
2    pickup_longitude    0.009610
3     pickup_latitude    0.008865
0     passenger_count    0.001673
11       is_rush_hour    0.001196
7                 day    0.000000
8               month    0.000000
✓ Random Forest Training Complete
  RMSE: 2.8280
  MAE:  1.7217
  R²:   0.9300

Top 10 Most Important Features (Random Forest):
              feature  importance
1       trip_distance    0.939264
6                hour    0.013637
5    dropoff_latitude    0.013274
4   dropoff_longitude    0.012480
2    pickup_longitude    0.009610
3     pickup_latitude    0.008865
0     passenger_count    0.001673
11       is_rush_hour    0.001

In [13]:
# ==================== MODEL 3: Gradient Boosting Regressor ====================
print("\n--- Model 3: Gradient Boosting Regressor ---")
gb_reg = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    validation_fraction=0.2,
    n_iter_no_change=10,
    verbose=0
)

print("Training Gradient Boosting...")
gb_reg.fit(X_train, y_train)
y_pred_gb = gb_reg.predict(X_test)

rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))
mae_gb = mean_absolute_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"✓ Gradient Boosting Training Complete")
print(f"  RMSE: {rmse_gb:.4f}")
print(f"  MAE:  {mae_gb:.4f}")
print(f"  R²:   {r2_gb:.4f}")

# Extract feature importance for GB
feature_importance_gb = pd.DataFrame({
    'feature': feature_cols,
    'importance': gb_reg.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features (Gradient Boosting):")
print(feature_importance_gb.head(10))


--- Model 3: Gradient Boosting Regressor ---
Training Gradient Boosting...
✓ Gradient Boosting Training Complete
  RMSE: 2.6636
  MAE:  1.6559
  R²:   0.9379

Top 10 Most Important Features (Gradient Boosting):
              feature  importance
1       trip_distance    0.962251
6                hour    0.011883
4   dropoff_longitude    0.008182
5    dropoff_latitude    0.008131
2    pickup_longitude    0.004503
3     pickup_latitude    0.003943
11       is_rush_hour    0.000828
0     passenger_count    0.000279
7                 day    0.000000
8               month    0.000000
✓ Gradient Boosting Training Complete
  RMSE: 2.6636
  MAE:  1.6559
  R²:   0.9379

Top 10 Most Important Features (Gradient Boosting):
              feature  importance
1       trip_distance    0.962251
6                hour    0.011883
4   dropoff_longitude    0.008182
5    dropoff_latitude    0.008131
2    pickup_longitude    0.004503
3     pickup_latitude    0.003943
11       is_rush_hour    0.000828
0     

In [15]:
# ==================== MODELS 4-6: Linear Regression & Regularized Models ====================
print("\n--- Models 4-6: Linear Regression, Ridge, and Lasso ---")

# Standardize features for regularized models
print("Standardizing features for regularized models...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("✓ Features standardized")

# MODEL 4: Linear Regression (Baseline)
print("\nTraining Linear Regression (Baseline)...")
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
y_pred_lr = lr_reg.predict(X_test)

rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print(f"✓ Linear Regression Training Complete")
print(f"  RMSE: {rmse_lr:.4f}")
print(f"  MAE:  {mae_lr:.4f}")
print(f"  R²:   {r2_lr:.4f}")

# MODEL 5: Ridge Regression (L2 Regularization)
print("\nTraining Ridge Regression (L2)...")
ridge_reg = Ridge(alpha=1.0)
ridge_reg.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_reg.predict(X_test_scaled)

rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"✓ Ridge Regression Training Complete")
print(f"  RMSE: {rmse_ridge:.4f}")
print(f"  MAE:  {mae_ridge:.4f}")
print(f"  R²:   {r2_ridge:.4f}")

# MODEL 6: Lasso Regression (L1 Regularization)
print("\nTraining Lasso Regression (L1)...")
lasso_reg = Lasso(alpha=0.01)
lasso_reg.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_reg.predict(X_test_scaled)

rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"✓ Lasso Regression Training Complete")
print(f"  RMSE: {rmse_lasso:.4f}")
print(f"  MAE:  {mae_lasso:.4f}")
print(f"  R²:   {r2_lasso:.4f}")

print("\n" + "="*70)
print("✓ ALL 6 MODELS TRAINED SUCCESSFULLY")
print("="*70)


--- Models 4-6: Linear Regression, Ridge, and Lasso ---
Standardizing features for regularized models...
✓ Features standardized

Training Linear Regression (Baseline)...
✓ Linear Regression Training Complete
  RMSE: 3.7173
  MAE:  2.3575
  R²:   0.8790

Training Ridge Regression (L2)...
✓ Ridge Regression Training Complete
  RMSE: 3.7173
  MAE:  2.3575
  R²:   0.8790

Training Lasso Regression (L1)...
✓ Lasso Regression Training Complete
  RMSE: 3.7143
  MAE:  2.3554
  R²:   0.8792

✓ ALL 6 MODELS TRAINED SUCCESSFULLY


In [16]:
# ==================== SECTION 4: ADVANCED METRICS (MAPE & QUANTILE LOSS) ====================
print("\n" + "="*70)
print("SECTION 4: ADVANCED EVALUATION METRICS")
print("="*70)

# Define quantile (pinball) loss
def quantile_loss(y_true, y_pred, q=0.5):
    resid = y_true - y_pred
    return np.mean(np.maximum(q * resid, (q - 1) * resid))

# Compute metrics for each model
metrics_dict = {}

metrics_dict['XGBoost'] = {
    'RMSE': float(rmse_xgb),
    'MAE': float(mae_xgb),
    'R2': float(r2_xgb),
    'MAPE': float(mean_absolute_percentage_error(y_test, y_pred_xgb)),
    'Quantile_Loss_0.25': float(quantile_loss(y_test.values, y_pred_xgb, 0.25)),
    'Quantile_Loss_0.5': float(quantile_loss(y_test.values, y_pred_xgb, 0.5)),
    'Quantile_Loss_0.75': float(quantile_loss(y_test.values, y_pred_xgb, 0.75))
}

metrics_dict['Random Forest'] = {
    'RMSE': float(rmse_rf),
    'MAE': float(mae_rf),
    'R2': float(r2_rf),
    'MAPE': float(mean_absolute_percentage_error(y_test, y_pred_rf)),
    'Quantile_Loss_0.25': float(quantile_loss(y_test.values, y_pred_rf, 0.25)),
    'Quantile_Loss_0.5': float(quantile_loss(y_test.values, y_pred_rf, 0.5)),
    'Quantile_Loss_0.75': float(quantile_loss(y_test.values, y_pred_rf, 0.75))
}

metrics_dict['Gradient Boosting'] = {
    'RMSE': float(rmse_gb),
    'MAE': float(mae_gb),
    'R2': float(r2_gb),
    'MAPE': float(mean_absolute_percentage_error(y_test, y_pred_gb)),
    'Quantile_Loss_0.25': float(quantile_loss(y_test.values, y_pred_gb, 0.25)),
    'Quantile_Loss_0.5': float(quantile_loss(y_test.values, y_pred_gb, 0.5)),
    'Quantile_Loss_0.75': float(quantile_loss(y_test.values, y_pred_gb, 0.75))
}

metrics_dict['Linear Regression'] = {
    'RMSE': float(rmse_lr),
    'MAE': float(mae_lr),
    'R2': float(r2_lr),
    'MAPE': float(mean_absolute_percentage_error(y_test, y_pred_lr)),
    'Quantile_Loss_0.25': float(quantile_loss(y_test.values, y_pred_lr, 0.25)),
    'Quantile_Loss_0.5': float(quantile_loss(y_test.values, y_pred_lr, 0.5)),
    'Quantile_Loss_0.75': float(quantile_loss(y_test.values, y_pred_lr, 0.75))
}

metrics_dict['Ridge Regression'] = {
    'RMSE': float(rmse_ridge),
    'MAE': float(mae_ridge),
    'R2': float(r2_ridge),
    'MAPE': float(mean_absolute_percentage_error(y_test, y_pred_ridge)),
    'Quantile_Loss_0.25': float(quantile_loss(y_test.values, y_pred_ridge, 0.25)),
    'Quantile_Loss_0.5': float(quantile_loss(y_test.values, y_pred_ridge, 0.5)),
    'Quantile_Loss_0.75': float(quantile_loss(y_test.values, y_pred_ridge, 0.75))
}

metrics_dict['Lasso Regression'] = {
    'RMSE': float(rmse_lasso),
    'MAE': float(mae_lasso),
    'R2': float(r2_lasso),
    'MAPE': float(mean_absolute_percentage_error(y_test, y_pred_lasso)),
    'Quantile_Loss_0.25': float(quantile_loss(y_test.values, y_pred_lasso, 0.25)),
    'Quantile_Loss_0.5': float(quantile_loss(y_test.values, y_pred_lasso, 0.5)),
    'Quantile_Loss_0.75': float(quantile_loss(y_test.values, y_pred_lasso, 0.75))
}

# Convert to DataFrame for easy viewing
metrics_df = pd.DataFrame(metrics_dict).T
print("\n=== Advanced Metrics Comparison ===\n")
print(metrics_df.round(6))

# Save metrics to a JSON file
with open('phase2_advanced_metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=4)

print("\n✓ Advanced metrics saved to 'phase2_advanced_metrics.json'")


SECTION 4: ADVANCED EVALUATION METRICS

=== Advanced Metrics Comparison ===

                       RMSE       MAE        R2      MAPE  Quantile_Loss_0.25  \
XGBoost            2.774382  1.661356  0.932612  0.115553            0.935651   
Random Forest      2.828006  1.721698  0.929982  0.120531            1.027025   
Gradient Boosting  2.663600  1.655924  0.937886  0.115174            0.937920   
Linear Regression  3.717276  2.357502  0.879023  0.191585            1.546342   
Ridge Regression   3.717264  2.357518  0.879024  0.191589            1.546342   
Lasso Regression   3.714251  2.355394  0.879220  0.191462            1.543073   

                   Quantile_Loss_0.5  Quantile_Loss_0.75  
XGBoost                     0.830678            0.725706  
Random Forest               0.860849            0.694673  
Gradient Boosting           0.827962            0.718004  
Linear Regression           1.178751            0.811161  
Ridge Regression            1.178759            0.811176  


In [17]:
# ==================== SECTION 5: SPATIAL-TEMPORAL CROSS-VALIDATION ====================
print("\n" + "="*70)
print("SECTION 5: SPATIAL-TEMPORAL CROSS-VALIDATION (5 folds)")
print("="*70)

# Combine train and test for CV (ensure data is time-ordered for TimeSeriesSplit)
full_data = pd.concat([train_df, test_df], ignore_index=True)
X_full = full_data[feature_cols]
y_full = full_data[target_col]

# TimeSeriesSplit
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

cv_results = {
    'fold': [],
    'xgb_rmse': [],
    'rf_rmse': [],
    'gb_rmse': [],
    'lr_rmse': [],
    'ridge_rmse': [],
    'lasso_rmse': []
}

print(f"Running TimeSeriesSplit with {n_splits} folds on {len(X_full)} samples...\n")

for i, (train_idx, test_idx) in enumerate(tscv.split(X_full), start=1):
    print(f"Fold {i}: train={len(train_idx)} samples, test={len(test_idx)} samples")
    X_tr, X_val = X_full.iloc[train_idx], X_full.iloc[test_idx]
    y_tr, y_val = y_full.iloc[train_idx], y_full.iloc[test_idx]

    # XGBoost
    xgb_cv = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, n_jobs=-1, verbosity=0)
    xgb_cv.fit(X_tr, y_tr)
    y_pred_xgb_cv = xgb_cv.predict(X_val)
    rmse_xgb_cv = np.sqrt(mean_squared_error(y_val, y_pred_xgb_cv))

    # Random Forest
    rf_cv = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
    rf_cv.fit(X_tr, y_tr)
    y_pred_rf_cv = rf_cv.predict(X_val)
    rmse_rf_cv = np.sqrt(mean_squared_error(y_val, y_pred_rf_cv))

    # Gradient Boosting
    gb_cv = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    gb_cv.fit(X_tr, y_tr)
    y_pred_gb_cv = gb_cv.predict(X_val)
    rmse_gb_cv = np.sqrt(mean_squared_error(y_val, y_pred_gb_cv))

    # Linear Regression
    lr_cv = LinearRegression()
    lr_cv.fit(X_tr, y_tr)
    y_pred_lr_cv = lr_cv.predict(X_val)
    rmse_lr_cv = np.sqrt(mean_squared_error(y_val, y_pred_lr_cv))

    # Ridge & Lasso require scaling
    scaler_cv = StandardScaler()
    X_tr_s = scaler_cv.fit_transform(X_tr)
    X_val_s = scaler_cv.transform(X_val)

    ridge_cv = Ridge(alpha=1.0)
    ridge_cv.fit(X_tr_s, y_tr)
    y_pred_ridge_cv = ridge_cv.predict(X_val_s)
    rmse_ridge_cv = np.sqrt(mean_squared_error(y_val, y_pred_ridge_cv))

    lasso_cv = Lasso(alpha=0.01)
    lasso_cv.fit(X_tr_s, y_tr)
    y_pred_lasso_cv = lasso_cv.predict(X_val_s)
    rmse_lasso_cv = np.sqrt(mean_squared_error(y_val, y_pred_lasso_cv))

    # Record results
    cv_results['fold'].append(i)
    cv_results['xgb_rmse'].append(float(rmse_xgb_cv))
    cv_results['rf_rmse'].append(float(rmse_rf_cv))
    cv_results['gb_rmse'].append(float(rmse_gb_cv))
    cv_results['lr_rmse'].append(float(rmse_lr_cv))
    cv_results['ridge_rmse'].append(float(rmse_ridge_cv))
    cv_results['lasso_rmse'].append(float(rmse_lasso_cv))

    print(f"  XGBoost RMSE: {rmse_xgb_cv:.4f}")
    print(f"  Random Forest RMSE: {rmse_rf_cv:.4f}")
    print(f"  Gradient Boosting RMSE: {rmse_gb_cv:.4f}")
    print(f"  Linear Regression RMSE: {rmse_lr_cv:.4f}")
    print(f"  Ridge RMSE: {rmse_ridge_cv:.4f}")
    print(f"  Lasso RMSE: {rmse_lasso_cv:.4f}\n")

# Summarize CV results
cv_df = pd.DataFrame(cv_results)
summary = {}
for model_label, col in [('XGBoost','xgb_rmse'), ('Random Forest','rf_rmse'), ('Gradient Boosting','gb_rmse'), ('Linear Regression','lr_rmse'), ('Ridge','ridge_rmse'), ('Lasso','lasso_rmse')]:
    vals = cv_df[col]
    summary[model_label] = {
        'mean_rmse': float(vals.mean()),
        'std_rmse': float(vals.std()),
        'fold_rmse': list(map(float, vals.tolist()))
    }

print("\n=== Cross-Validation Summary ===\n")
print(pd.DataFrame(summary).T.round(4))

# Save CV results
with open('phase2_cv_results.json', 'w') as f:
    json.dump({'cv_summary': summary, 'cv_folds': cv_results}, f, indent=4)

print("\n✓ Cross-validation results saved to 'phase2_cv_results.json'")

# Store cv_df for downstream plotting
cv_results_df = cv_df



SECTION 5: SPATIAL-TEMPORAL CROSS-VALIDATION (5 folds)
Running TimeSeriesSplit with 5 folds on 96267 samples...

Fold 1: train=16047 samples, test=16044 samples
  XGBoost RMSE: 3.0506
  Random Forest RMSE: 3.1526
  Gradient Boosting RMSE: 3.1033
  Linear Regression RMSE: 4.6122
  Ridge RMSE: 4.6122
  Lasso RMSE: 4.5830

Fold 2: train=32091 samples, test=16044 samples
  XGBoost RMSE: 3.0506
  Random Forest RMSE: 3.1526
  Gradient Boosting RMSE: 3.1033
  Linear Regression RMSE: 4.6122
  Ridge RMSE: 4.6122
  Lasso RMSE: 4.5830

Fold 2: train=32091 samples, test=16044 samples
  XGBoost RMSE: 3.0207
  Random Forest RMSE: 3.0379
  Gradient Boosting RMSE: 3.0414
  Linear Regression RMSE: 3.8714
  Ridge RMSE: 3.8715
  Lasso RMSE: 3.8719

Fold 3: train=48135 samples, test=16044 samples
  XGBoost RMSE: 3.0207
  Random Forest RMSE: 3.0379
  Gradient Boosting RMSE: 3.0414
  Linear Regression RMSE: 3.8714
  Ridge RMSE: 3.8715
  Lasso RMSE: 3.8719

Fold 3: train=48135 samples, test=16044 samples
  

## Section 6: SHAP Analysis & Model Interpretability

This section computes SHAP values for the primary model (XGBoost) and generates visualizations to explain feature contributions to predictions.

In [18]:
# ==================== SECTION 6: SHAP ANALYSIS & INTERPRETABILITY ====================
print("\n" + "="*70)
print("SECTION 6: SHAP ANALYSIS & MODEL INTERPRETABILITY")
print("="*70)

# Ensure SHAP is installed and importable
import shap


# Compute SHAP values for XGBoost model
print("\nComputing SHAP values for XGBoost model (this may take some time)...")
try:
    explainer = shap.TreeExplainer(xgb_reg)
    shap_values = explainer.shap_values(X_test)
except Exception:
    # Fallback to the newer API
    explainer = shap.Explainer(xgb_reg, X_train)
    shap_exp = explainer(X_test)
    try:
        shap_values = shap_exp.values
    except Exception:
        shap_values = shap_exp

print("SHAP values computed.")

# Create and save SHAP visualizations
import matplotlib.pyplot as plt
plt.switch_backend('agg')

# 1) SHAP summary (bar)
print("Generating SHAP summary (bar) plot...")
plt.figure(figsize=(10, 8))
try:
    shap.summary_plot(shap_values, X_test, plot_type='bar', show=False)
    plt.title('SHAP Summary - Feature Importance (XGBoost)')
    plt.tight_layout()
    plt.savefig('shap_summary_bar.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved 'shap_summary_bar.png'")
except Exception as e:
    print(f"Could not create SHAP summary bar plot: {e}")

# 2) SHAP beeswarm (impact distribution)
print("Generating SHAP beeswarm plot...")
plt.figure(figsize=(12, 8))
try:
    shap.summary_plot(shap_values, X_test, show=False)
    plt.title('SHAP Summary - Feature Impact (XGBoost)')
    plt.tight_layout()
    plt.savefig('shap_summary_beeswarm.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved 'shap_summary_beeswarm.png'")
except Exception as e:
    print(f"Could not create SHAP beeswarm plot: {e}")

# 3) SHAP dependence plots for top features
print("Generating SHAP dependence plots for top features...")
try:
    top_feats = feature_importance_gb.head(4)['feature'].values.tolist()
except Exception:
    # Fall back to RF importance if GB not available
    top_feats = feature_importance_rf.head(4)['feature'].values.tolist()

saved_dependence = []
for feat in top_feats:
    try:
        plt.figure(figsize=(8, 6))
        shap.dependence_plot(feat, shap_values, X_test, show=False)
        fname = f'shap_dependence_{feat}.png'
        plt.title(f'SHAP Dependence - {feat}')
        plt.tight_layout()
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
        saved_dependence.append(fname)
    except Exception as e:
        print(f"Could not create dependence plot for {feat}: {e}")

if saved_dependence:
    print("Saved dependence plots:")
    for p in saved_dependence:
        print(f"  - {p}")
else:
    print("No dependence plots were saved.")

print("\nSHAP analysis completed. Plots saved to working directory.")


SECTION 6: SHAP ANALYSIS & MODEL INTERPRETABILITY


c:\Users\omar8\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SHAP library loaded.

Computing SHAP values for XGBoost model (this may take some time)...
SHAP values computed.
Generating SHAP summary (bar) plot...
SHAP values computed.
Generating SHAP summary (bar) plot...
Saved 'shap_summary_bar.png'
Generating SHAP beeswarm plot...
Saved 'shap_summary_bar.png'
Generating SHAP beeswarm plot...
Saved 'shap_summary_beeswarm.png'
Generating SHAP dependence plots for top features...
Saved 'shap_summary_beeswarm.png'
Generating SHAP dependence plots for top features...
Saved dependence plots:
  - shap_dependence_trip_distance.png
  - shap_dependence_hour.png
  - shap_dependence_dropoff_longitude.png
  - shap_dependence_dropoff_latitude.png

SHAP analysis completed. Plots saved to working directory.
Saved dependence plots:
  - shap_dependence_trip_distance.png
  - shap_dependence_hour.png
  - shap_dependence_dropoff_longitude.png
  - shap_dependence_dropoff_latitude.png

SHAP analysis completed. Plots saved to working directory.


In [19]:
# ==================== SECTION 7: PERFORMANCE BY TEMPORAL FEATURES ====================
print("\n" + "="*70)
print("SECTION 7: PERFORMANCE BY TEMPORAL FEATURES")
print("="*70)

# Prepare test Analysis DF
test_analysis = X_test.copy()
if 'actual' not in test_analysis.columns:
    test_analysis['actual'] = y_test.values
# Add predictions from models (if present)
model_preds = {
    'xgb_pred': 'y_pred_xgb',
    'rf_pred': 'y_pred_rf',
    'gb_pred': 'y_pred_gb',
    'lr_pred': 'y_pred_lr',
    'ridge_pred': 'y_pred_ridge',
    'lasso_pred': 'y_pred_lasso'
}
for col_name, var_name in model_preds.items():
    if var_name in globals():
        test_analysis[col_name] = globals()[var_name]

# Helper to compute metrics
def group_metrics(df, actual_col, pred_col):
    return pd.Series({
        'RMSE': np.sqrt(mean_squared_error(df[actual_col], df[pred_col])),
        'MAE': mean_absolute_error(df[actual_col], df[pred_col]),
        'R2': r2_score(df[actual_col], df[pred_col]),
        'Count': len(df)
    })

# Choose temporal columns to evaluate
temporal_cols = [c for c in ['hour','day','is_rush_hour','is_weekend'] if c in test_analysis.columns]

for col in temporal_cols:
    print(f"\n--- Performance by {col} ---")
    results = []
    groups = test_analysis.groupby(col, observed=True)
    for name, grp in groups:
        row = {'group': name}
        for model_col in ['xgb_pred','rf_pred','gb_pred','lr_pred','ridge_pred','lasso_pred']:
            if model_col in grp.columns:
                m = group_metrics(grp, 'actual', model_col)
                row[f"{model_col}_RMSE"] = m['RMSE']
                row[f"{model_col}_MAE"] = m['MAE']
                row[f"{model_col}_R2"] = m['R2']
        row['count'] = len(grp)
        results.append(row)
    res_df = pd.DataFrame(results).sort_values('group')
    print(res_df.round(4))
    csv_name = f'temporal_performance_by_{col}.csv'
    res_df.to_csv(csv_name, index=False)
    print(f"Saved {csv_name}")

    # Plot RMSE heatmap/bar for available models
    rmse_cols = [c for c in res_df.columns if c.endswith('_RMSE')]
    if len(rmse_cols) > 0:
        plt.figure(figsize=(10,6))
        # plot RMSE for first three models to keep plot readable
        plot_cols = rmse_cols[:6]
        res_plot = res_df.set_index('group')[plot_cols]
        res_plot.plot(kind='bar', figsize=(12,6))
        plt.title(f'RMSE by {col}')
        plt.ylabel('RMSE')
        plt.xlabel(col)
        plt.xticks(rotation=45)
        plt.tight_layout()
        img_name = f'temporal_rmse_by_{col}.png'
        plt.savefig(img_name, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved plot {img_name}")

print('\n✓ Temporal performance analysis complete.')


SECTION 7: PERFORMANCE BY TEMPORAL FEATURES

--- Performance by hour ---
    group  xgb_pred_RMSE  xgb_pred_MAE  xgb_pred_R2  rf_pred_RMSE  \
0  0.7778         2.7892        1.6508       0.9381        3.0147   
1  0.8889         2.6505        1.5552       0.9378        2.7404   
2  1.0000         2.6563        1.6309       0.9360        2.7573   
3  1.1111         3.0194        1.8010       0.9238        2.9821   

   rf_pred_MAE  rf_pred_R2  gb_pred_RMSE  gb_pred_MAE  gb_pred_R2  \
0       1.7373      0.9277        2.4346       1.5990      0.9529   
1       1.6206      0.9335        2.6797       1.5722      0.9364   
2       1.6826      0.9310        2.5029       1.6214      0.9431   
3       1.8657      0.9257        2.8409       1.7811      0.9326   

   lr_pred_RMSE  lr_pred_MAE  lr_pred_R2  ridge_pred_RMSE  ridge_pred_MAE  \
0        3.4957       2.2743      0.9028           3.4957          2.2743   
1        3.1280       2.1723      0.9133           3.1280          2.1723   
2  

In [21]:
# ==================== SECTION 8: SPATIAL PERFORMANCE ANALYSIS ====================
print('\n' + '='*70)
print('SECTION 8: SPATIAL PERFORMANCE ANALYSIS')
print('='*70)

# Ensure 'test_analysis' exists (created in Section 7) and the helper 'group_metrics' is available
import matplotlib.pyplot as plt

try:
    group_metrics
except NameError:
    def group_metrics(df, actual_col, pred_col):
        return pd.Series({
            'RMSE': np.sqrt(mean_squared_error(df[actual_col], df[pred_col])) ,
            'MAE': mean_absolute_error(df[actual_col], df[pred_col]),
            'R2': r2_score(df[actual_col], df[pred_col]),
            'Count': len(df)
        })

if 'test_analysis' not in globals():
    test_analysis = X_test.copy()
    test_analysis['actual'] = y_test.values
    model_preds = {
        'xgb_pred': 'y_pred_xgb',
        'rf_pred': 'y_pred_rf',
        'gb_pred': 'y_pred_gb',
        'lr_pred': 'y_pred_lr',
        'ridge_pred': 'y_pred_ridge',
        'lasso_pred': 'y_pred_lasso'
    }
    for col_name, var_name in model_preds.items():
        if var_name in globals():
            test_analysis[col_name] = globals()[var_name]

# 1) Performance by trip_distance categories
if 'trip_distance' in test_analysis.columns:
    print('\nAnalyzing performance by trip distance categories...')
    # Define bins; fall back to quantiles if necessary
    try:
        bins = [0, 1, 3, 10, np.inf]
        labels = ['Very Short', 'Short', 'Medium', 'Long']
        test_analysis['distance_cat'] = pd.cut(test_analysis['trip_distance'], bins=bins, labels=labels, include_lowest=True)
    except Exception:
        test_analysis['distance_cat'] = pd.qcut(test_analysis['trip_distance'], 4, labels=['Very Short','Short','Medium','Long'])

    results = []
    groups = test_analysis.groupby('distance_cat', observed=True)
    for name, grp in groups:
        row = {'group': str(name)}
        for model_col in ['xgb_pred','rf_pred','gb_pred','lr_pred','ridge_pred','lasso_pred']:
            if model_col in grp.columns:
                m = group_metrics(grp, 'actual', model_col)
                row[f'{model_col}_RMSE'] = m['RMSE']
                row[f'{model_col}_MAE'] = m['MAE']
                row[f'{model_col}_R2'] = m['R2']
        row['count'] = len(grp)
        results.append(row)
    res_df = pd.DataFrame(results).sort_values('group')
    csv_name = 'spatial_performance_by_distance.csv'
    res_df.to_csv(csv_name, index=False)
    print(f'Saved {csv_name}')

    # Plot RMSE by distance category
    rmse_cols = [c for c in res_df.columns if c.endswith('_RMSE')]
    if len(rmse_cols) > 0:
        plt.figure(figsize=(10,6))
        plot_df = res_df.set_index('group')[rmse_cols]
        plot_df.plot(kind='bar', figsize=(10,6))
        plt.title('RMSE by Trip Distance Category')
        plt.ylabel('RMSE')
        plt.xlabel('Distance Category')
        plt.xticks(rotation=0)
        plt.tight_layout()
        img_name = 'spatial_rmse_by_distance.png'
        plt.savefig(img_name, dpi=300, bbox_inches='tight')
        plt.close()
        print(f'Saved plot {img_name}')
else:
    print('trip_distance column not found in test data; skipping distance-based analysis.')

# 2) Performance by passenger_count
if 'passenger_count' in test_analysis.columns:
    print('\nAnalyzing performance by passenger_count...')
    test_analysis['passenger_count_group'] = test_analysis['passenger_count'].clip(lower=0).astype(int)
    results = []
    groups = test_analysis.groupby('passenger_count_group', observed=True)
    for name, grp in groups:
        row = {'group': int(name)}
        for model_col in ['xgb_pred','rf_pred','gb_pred','lr_pred','ridge_pred','lasso_pred']:
            if model_col in grp.columns:
                m = group_metrics(grp, 'actual', model_col)
                row[f'{model_col}_RMSE'] = m['RMSE']
                row[f'{model_col}_MAE'] = m['MAE']
                row[f'{model_col}_R2'] = m['R2']
        row['count'] = len(grp)
        results.append(row)
    res_df_pc = pd.DataFrame(results).sort_values('group')
    csv_name_pc = 'spatial_performance_by_passenger_count.csv'
    res_df_pc.to_csv(csv_name_pc, index=False)
    print(f'Saved {csv_name_pc}')

    rmse_cols = [c for c in res_df_pc.columns if c.endswith('_RMSE')]
    if len(rmse_cols) > 0:
        plt.figure(figsize=(10,6))
        plot_df = res_df_pc.set_index('group')[rmse_cols]
        plot_df.plot(kind='bar', figsize=(10,6))
        plt.title('RMSE by Passenger Count')
        plt.ylabel('RMSE')
        plt.xlabel('Passenger Count')
        plt.xticks(rotation=0)
        plt.tight_layout()
        img_name_pc = 'spatial_rmse_by_passenger_count.png'
        plt.savefig(img_name_pc, dpi=300, bbox_inches='tight')
        plt.close()
        print(f'Saved plot {img_name_pc}')
else:
    print('passenger_count column not found in test data; skipping passenger_count analysis.')

print('\n✓ Spatial performance analysis complete.')


SECTION 8: SPATIAL PERFORMANCE ANALYSIS

Analyzing performance by trip distance categories...
Saved spatial_performance_by_distance.csv
Saved plot spatial_rmse_by_distance.png

Analyzing performance by passenger_count...
Saved spatial_performance_by_passenger_count.csv
Saved plot spatial_rmse_by_distance.png

Analyzing performance by passenger_count...
Saved spatial_performance_by_passenger_count.csv
Saved plot spatial_rmse_by_passenger_count.png

✓ Spatial performance analysis complete.
Saved plot spatial_rmse_by_passenger_count.png

✓ Spatial performance analysis complete.


In [22]:
# ==================== SECTION 9: VISUALIZATION CONSOLIDATION & COMPARISON PLOTS ====================
print('\n' + '='*70)
print('SECTION 9: VISUALIZATION CONSOLIDATION & COMPARISON PLOTS')
print('='*70)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
plt.switch_backend('agg')

# Ensure metrics_df exists
if 'metrics_df' not in globals():
    raise RuntimeError('metrics_df not found — run Section 4 first to compute metrics.')

# 1) Bar charts: RMSE, MAE, R2, MAPE across models
metrics_to_plot = ['RMSE','MAE','R2','MAPE']
for metric in metrics_to_plot:
    try:
        vals = metrics_df[metric].dropna()
    except Exception as e:
        print(f"Skipping {metric}: not available ({e})")
        continue
    plt.figure(figsize=(8,5))
    vals.sort_values().plot(kind='bar', color='C0')
    plt.title(f'{metric} Comparison Across Models')
    plt.ylabel(metric)
    plt.xlabel('Model')
    plt.xticks(rotation=45)
    plt.tight_layout()
    fname = f'comparison_{metric.lower()}.png'
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved {fname}")

# 2) Feature importance comparison (RF vs GB) — top 10 union
fi_rf = feature_importance_rf.copy() if 'feature_importance_rf' in globals() else None
fi_gb = feature_importance_gb.copy() if 'feature_importance_gb' in globals() else None
if fi_rf is None and fi_gb is None:
    print('No feature importances available (RF/GB missing). Skipping feature importance comparison.')
else:
    # prepare top features union
    top_rf = fi_rf.head(10) if fi_rf is not None else pd.DataFrame(columns=['feature','importance'])
    top_gb = fi_gb.head(10) if fi_gb is not None else pd.DataFrame(columns=['feature','importance'])
    union_feats = pd.Index(top_rf['feature'].tolist() + top_gb['feature'].tolist()).unique()
    comp_df = pd.DataFrame({'feature': union_feats})
    if fi_rf is not None:
        comp_df = comp_df.merge(fi_rf[['feature','importance']].rename(columns={'importance':'rf_imp'}), on='feature', how='left')
    else:
        comp_df['rf_imp'] = 0.0
    if fi_gb is not None:
        comp_df = comp_df.merge(fi_gb[['feature','importance']].rename(columns={'importance':'gb_imp'}), on='feature', how='left')
    else:
        comp_df['gb_imp'] = 0.0
    comp_df = comp_df.fillna(0)
    # Normalize for comparison
    for col in ['rf_imp','gb_imp']:
        if comp_df[col].sum() > 0:
            comp_df[col] = comp_df[col] / comp_df[col].sum()
    comp_df = comp_df.sort_values(['rf_imp','gb_imp'], ascending=False).set_index('feature')
    plt.figure(figsize=(10,6))
    comp_df.plot(kind='bar', figsize=(12,6))
    plt.title('Feature Importance Comparison (RF vs GB) — normalized')
    plt.ylabel('Normalized Importance')
    plt.xlabel('Feature')
    plt.xticks(rotation=45)
    plt.tight_layout()
    fname = 'feature_importance_rf_vs_gb.png'
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved {fname}")

# 3) CV RMSE boxplot (from cv_results_df / cv_df)
cv_source = None
if 'cv_results_df' in globals():
    cv_source = cv_results_df.copy()
elif 'cv_df' in globals():
    cv_source = cv_df.copy()
if cv_source is None or cv_source.empty:
    print('No CV results found. Skipping CV RMSE boxplot.')
else:
    # Melt folds into long format for boxplot
    cv_long = pd.melt(cv_source, id_vars=['fold'], var_name='model_rmse', value_name='rmse')
    # keep only rmse columns
    cv_long = cv_long[cv_long['model_rmse'].str.contains('_rmse', case=False)]
    # map model labels (xgb_rmse -> XGBoost)
    cv_long['model'] = cv_long['model_rmse'].str.replace('_rmse','').str.replace('_',' ').str.title()
    plt.figure(figsize=(10,6))
    sns.boxplot(x='model', y='rmse', data=cv_long)
    plt.title('Cross-Validation RMSE Distribution by Model')
    plt.ylabel('RMSE')
    plt.xlabel('Model')
    plt.xticks(rotation=45)
    plt.tight_layout()
    fname = 'cv_rmse_boxplot.png'
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved {fname}")

# 4) Prediction vs Actual scatter plots for top models (XGBoost, Random Forest, Gradient Boosting if available)
top_models = [('XGBoost','y_pred_xgb'), ('Random Forest','y_pred_rf'), ('Gradient Boosting','y_pred_gb')]
for label, pred_var in top_models:
    if pred_var in globals():
        plt.figure(figsize=(6,6))
        plt.scatter(test_analysis['actual'], globals()[pred_var], alpha=0.4, s=10)
        plt.plot([test_analysis['actual'].min(), test_analysis['actual'].max()],[test_analysis['actual'].min(), test_analysis['actual'].max()], 'r--')
        plt.xlabel('Actual')
        plt.ylabel('Predicted')
        plt.title(f'Predicted vs Actual — {label}')
        plt.tight_layout()
        fname = f'pred_vs_actual_{label.lower().replace(" ","_")}.png'
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved {fname}")
    else:
        print(f"Prediction variable {pred_var} not found — skipping {label} scatter plot.")

print('\n✓ Section 9 visualizations created (files saved to working directory).')


SECTION 9: VISUALIZATION CONSOLIDATION & COMPARISON PLOTS
Saved comparison_rmse.png
Saved comparison_mae.png
Saved comparison_r2.png
Saved comparison_mape.png
Saved feature_importance_rf_vs_gb.png
Saved cv_rmse_boxplot.png
Saved pred_vs_actual_xgboost.png
Saved pred_vs_actual_random_forest.png
Saved pred_vs_actual_gradient_boosting.png

✓ Section 9 visualizations created (files saved to working directory).


In [ ]:
# ==================== SECTION 10: MODEL DEPLOYMENT & RECOMMENDATIONS ====================
print('\n' + '='*70)
print('SECTION 10: MODEL DEPLOYMENT & RECOMMENDATIONS')
print('='*70)

# 1) Model Performance Summary & Ranking
print('\n--- Model Performance Ranking ---')
ranking_df = metrics_df[['RMSE', 'MAE', 'R2', 'MAPE']].copy()
ranking_df['Overall_Score'] = (
    ranking_df['RMSE'].rank() +
    ranking_df['MAE'].rank() +
    (6 - ranking_df['R2'].rank()) +
    ranking_df['MAPE'].rank()
)
ranking_df = ranking_df.sort_values('Overall_Score')
print(ranking_df.round(4))

# 2) Determine Best Model
best_model_name = ranking_df.index[0]
best_model = None
if best_model_name == 'XGBoost':
    best_model = xgb_reg
elif best_model_name == 'Random Forest':
    best_model = rf_reg
elif best_model_name == 'Gradient Boosting':
    best_model = gb_reg
elif best_model_name == 'Linear Regression':
    best_model = lr_reg
elif best_model_name == 'Ridge Regression':
    best_model = ridge_reg
elif best_model_name == 'Lasso Regression':
    best_model = lasso_reg

print(f'\n✓ Best Model Selected: {best_model_name}')
print(f'  RMSE: {metrics_df.loc[best_model_name, "RMSE"]:.4f}')
print(f'  MAE:  {metrics_df.loc[best_model_name, "MAE"]:.4f}')
print(f'  R²:   {metrics_df.loc[best_model_name, "R2"]:.4f}')

# 3) Save Best Model
if best_model is not None:
    model_filename = f'{best_model_name.lower().replace(" ", "_")}_regressor_phase2.joblib'
    joblib.dump(best_model, model_filename)
    print(f'\n✓ Best model saved to: {model_filename}')

# 4) Generate Deployment Recommendations Report
recommendations = {
    'Best_Model': best_model_name,
    'Key_Metrics': {
        'RMSE': float(metrics_df.loc[best_model_name, 'RMSE']),
        'MAE': float(metrics_df.loc[best_model_name, 'MAE']),
        'R2': float(metrics_df.loc[best_model_name, 'R2']),
        'MAPE': float(metrics_df.loc[best_model_name, 'MAPE'])
    },
    'Model_Characteristics': {
        'Type': 'Tree-based Ensemble' if best_model_name in ['XGBoost', 'Random Forest', 'Gradient Boosting'] else 'Linear',
        'Interpretability': 'High' if best_model_name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression'] else 'Medium (SHAP available)',
        'Training_Speed': 'Fast' if best_model_name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression'] else 'Medium',
        'Inference_Speed': 'Very Fast',
        'Memory_Footprint': 'Small' if best_model_name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression'] else 'Medium'
    },
    'Deployment_Readiness': {
        'Model_File_Size_MB': os.path.getsize(model_filename) / (1024**2) if best_model is not None and os.path.exists(model_filename) else 'N/A',
        'Production_Ready': True,
        'Monitoring_Required': True,
        'Retraining_Frequency': 'Monthly or upon performance degradation'
    },
    'Strengths': [
        f'Best overall RMSE: {metrics_df.loc[best_model_name, "RMSE"]:.4f}',
        f'High R² score: {metrics_df.loc[best_model_name, "R2"]:.4f}',
        'Robust across cross-validation folds',
        'Handles non-linear relationships well'
    ],
    'Weaknesses': [
        'May overfit on edge cases (very long trips)',
        'Sensitive to outliers in target variable',
        'Requires careful feature engineering'
    ],
    'Recommendations': [
        '1. Deploy model in staging environment first',
        '2. Monitor prediction errors in production',
        '3. Set up automated retraining pipeline (monthly)',
        '4. Implement feature drift detection',
        '5. Create fallback to baseline model if performance degrades',
        '6. Use SHAP values for prediction explanations',
        '7. Track model performance metrics continuously',
        '8. Establish A/B testing for model updates'
    ]
}

# Save recommendations to JSON
with open('phase2_deployment_recommendations.json', 'w') as f:
    json.dump(recommendations, f, indent=4)

print('\n✓ Deployment recommendations saved to: phase2_deployment_recommendations.json')

# 5) Print Summary Report
print('\n' + '='*70)
print('DEPLOYMENT RECOMMENDATIONS SUMMARY')
print('='*70)
print(f'\nSelected Model: {recommendations["Best_Model"]}')
print('\nKey Metrics:')
for metric, value in recommendations['Key_Metrics'].items():
    print(f'  {metric}: {value:.4f}')
print('\nModel Characteristics:')
for char, value in recommendations['Model_Characteristics'].items():
    print(f'  {char}: {value}')
print('\nStrengths:')
for strength in recommendations['Strengths']:
    print(f'  • {strength}')
print('\nWeaknesses:')
for weakness in recommendations['Weaknesses']:
    print(f'  • {weakness}')
print('\nDeployment Recommendations:')
for rec in recommendations['Recommendations']:
    print(f'  {rec}')

# 6) Create final summary comparison table
comparison_summary = pd.DataFrame({
    'Model': metrics_df.index,
    'RMSE': metrics_df['RMSE'].values,
    'MAE': metrics_df['MAE'].values,
    'R2': metrics_df['R2'].values,
    'MAPE': metrics_df['MAPE'].values,
    'Production_Ready': 'Yes'
})

csv_filename = 'phase2_final_model_comparison.csv'
comparison_summary.to_csv(csv_filename, index=False)
print(f'\n✓ Final model comparison saved to: {csv_filename}')

print('\n' + '='*70)
print('✓ PHASE 2: SUPERVISED MACHINE LEARNING COMPLETE')
print('='*70)
print(f"\nTimestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nDeliverables:")
print("  1. phase2_advanced_metrics.json — Comprehensive metrics for all models")
print("  2. phase2_cv_results.json — Cross-validation results and fold-wise performance")
print("  3. phase2_deployment_recommendations.json — Deployment guidance and recommendations")
print("  4. phase2_final_model_comparison.csv — Final comparison table")
print("  5. Trained model file — Best model saved for production")
print("  6. Visualizations — Multiple PNG files for analysis and reporting")


SECTION 10: MODEL DEPLOYMENT & RECOMMENDATIONS

--- Model Performance Ranking ---
                     RMSE     MAE      R2    MAPE  Overall_Score
Gradient Boosting  2.6636  1.6559  0.9379  0.1152            3.0
XGBoost            2.7744  1.6614  0.9326  0.1156            7.0
Random Forest      2.8280  1.7217  0.9300  0.1205           11.0
Lasso Regression   3.7143  2.3554  0.8792  0.1915           15.0
Linear Regression  3.7173  2.3575  0.8790  0.1916           21.0
Ridge Regression   3.7173  2.3575  0.8790  0.1916           21.0

✓ Best Model Selected: Gradient Boosting
  RMSE: 2.6636
  MAE:  1.6559
  R²:   0.9379

✓ Best model saved to: gradient_boosting_regressor_phase2.joblib

✓ Deployment recommendations saved to: phase2_deployment_recommendations.json

DEPLOYMENT RECOMMENDATIONS SUMMARY

Selected Model: Gradient Boosting

Key Metrics:
  RMSE: 2.6636
  MAE: 1.6559
  R2: 0.9379
  MAPE: 0.1152

Model Characteristics:
  Type: Tree-based Ensemble
  Interpretability: Medium (SHAP ava